In [1]:
# Importamos librerias
import pandas as pd
import numpy as np

from datetime import datetime 

import os

import re

pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames
# Ignorar warings
import warnings
warnings.filterwarnings("ignore")

import dotenv
from dotenv import load_dotenv
load_dotenv()


pd.set_option('display.max_columns', None)

import sys 
sys.path.append(os.path.abspath("../src"))  

In [2]:
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors 

from psycopg2 import sql
import support_base_de_datos as sbd
import support_extraccion as se


En primer lugar cargamos los datafrmaes en los que basaremos nuestra inserción, que serán los 3 de información general delas carreras, los 3 de información de etapas y los 3 de información de puertos.

In [3]:
dicc_dataframes=se.cargar_datos_wikipedia()
[df_giro, df_tour, df_vuelta] = [dicc_dataframes["datos_giro"],
                                 dicc_dataframes["datos_tour"] ,
                                 dicc_dataframes["datos_vuelta"]]

In [4]:
df_giro.head()

,Edición,Año,Ganador,Tiempo del ganador,Kilómetros totales,Velocidad media,N.º de corredores que participaron,N.º de corredores que finalizaron,Puntos del ganador
0,1,1909,Luigi Ganna,NaN,2448.0,27.258,127,49,25
1,2,1910,Carlo Galetti,NaN,2987.5,26.114,101,22,28
2,3,1911,Carlo Galetti,NaN,3452.0,26.082,86,24,50
3,4,1912,Atala,NaN,2439.5,27.106,54,26,31
4,5,1913,Carlo Oriani,NaN,2932.0,26.379,99,35,37


In [5]:
lista_carreras=["giro-d-italia", "tour-de-france", "vuelta-a-espana"]

In [6]:
for carrera in lista_carreras:
    ruta_datos = '../datos/datos_api_procyclingstats/datos_limpiados'
    nombre_archivo = f'df_etapas_{carrera}.csv'
    ruta_archivo = os.path.join(ruta_datos, nombre_archivo)
    if carrera == "giro-d-italia":
        df_etapas_giro=pd.read_csv(ruta_archivo)
    elif carrera == "tour-de-france":
        df_etapas_tour=pd.read_csv(ruta_archivo)
    elif carrera == "vuelta-a-espana":
        df_etapas_vuelta=pd.read_csv(ruta_archivo)

In [7]:
df_puertos_giro=pd.read_csv('../datos/datos_api_geopy/datos_transformados/datos_puertos_giro.csv')
df_puertos_tour=pd.read_csv('../datos/datos_api_geopy/datos_transformados/datos_puertos_tour.csv')
df_puertos_vuelta_completo=pd.read_csv('../datos/datos_api_geopy/datos_transformados/datos_puertos_vuelta.csv')

### Creamos la base de datos y la conexion

In [8]:
sbd.crear_basedatos("localhost","postgres","admin","5432","Ciclismo")

Error de tipo: la base de datos «Ciclismo» ya existe



In [9]:
# Creamos la conexion
conn = psycopg2.connect(
    dbname="Ciclismo",
    user="postgres",
    password="admin",
    host="localhost",
    port="5432"
)
cursor = conn.cursor()


### Creamos las tablas

Para la creacion de tablas, he decidido hacer 3 tablas una para para las ediciones de las grandes vueltas, otra para las etapas de cada edición y otra para los puertos más frecuents. He tanteado la idea de cada una de estas tablas dividirla en 3 una para el giro, ota para el Tour y otra para la Vuela, pero eso complicaría bastante a la hora de hacer las queries así que lo he desechado.

Sobre la relación entre las tablas, la tabla de ediciones y la de etapas están conectadas mediante id_ediciones que es Primary Key de la tabla ediciones y foreign key de la tabla etapas. En cambio, para la tabla de puertos, dado que no tenemos información sobre en que edición se subió cada uno no hemso podido conectarlo con una primary key, sin embargo, el campo carrera conecta las 3 tablas lo que permite hacer consultas. 

In [10]:
cursor.execute("""
-- Tabla para las ediciones de las grandes vueltas
CREATE TABLE IF NOT EXISTS ediciones (
    id SERIAL PRIMARY KEY,
    carrera VARCHAR(20),
    edicion INT,
    ano INT,
    ganador VARCHAR(100),
    tiempo_ganador VARCHAR(50),
    kilometros_totales FLOAT,
    velocidad_media FLOAT,
    num_corredores_participantes INT,
    num_corredores_finalizaron INT,
    puntos_ganador INT
);

-- Tabla para las etapas de cada edición (sin el campo 'etapa')
CREATE TABLE IF NOT EXISTS etapas (
    id SERIAL PRIMARY KEY,
    carrera VARCHAR(20),
    fecha DATE,
    salida VARCHAR(100),
    llegada VARCHAR(100),
    distancia FLOAT,
    tipo_etapa VARCHAR(10),
    metros_verticales FLOAT,
    ganador_etapa VARCHAR(100),
    url_ganador VARCHAR(255),
    edad INT,
    nacionalidad VARCHAR(10),
    edicion_id INT REFERENCES ediciones(id) ON DELETE CASCADE
);

-- Tabla para los puertos más frecuentados
CREATE TABLE IF NOT EXISTS puertos (
    id SERIAL PRIMARY KEY,
    carrera VARCHAR(20),
    puerto VARCHAR(100),
    num_etapas INT,
    num_ediciones INT,
    ano_primera_vez INT,
    latitud FLOAT,
    longitud FLOAT
);
""")



### Inserción de datos

Usamos el metodo iterrows para iterar por filas. Hacemos las inserciones fijandonos en respetar la primay key y que los values se corresponden a los camos que queremos meter.

In [11]:
for carrera, df in zip(["Giro", "Tour", "Vuelta"], [df_giro, df_tour, df_vuelta]):
    for _, row in df.iterrows():
        cursor.execute("""
            INSERT INTO ediciones (carrera, edicion, ano, ganador, tiempo_ganador, kilometros_totales, velocidad_media,
                                   num_corredores_participantes, num_corredores_finalizaron, puntos_ganador)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            carrera,
            int(row['Edición']),
            int(row['Año']),
            row['Ganador'],
            row['Tiempo del ganador'] if pd.notna(row['Tiempo del ganador']) else None,
            row['Kilómetros totales'] if pd.notna(row['Kilómetros totales']) else None,
            row['Velocidad media'] if pd.notna(row['Velocidad media']) else None,
            int(row['N.º de corredores que participaron']) if pd.notna(row['N.º de corredores que participaron']) else None,
            int(row['N.º de corredores que finalizaron']) if pd.notna(row['N.º de corredores que finalizaron']) else None,
            int(row['Puntos del ganador']) if pd.notna(row['Puntos del ganador']) else None
        ))

# Recuperar los IDs de las ediciones insertadas para usarlos en las etapas
cursor.execute("SELECT id, carrera, edicion FROM ediciones")
ediciones_ids = cursor.fetchall()
ediciones_map = {(carrera, edicion): id for id, carrera, edicion in ediciones_ids}


for carrera, df in zip(["Giro", "Tour", "Vuelta"], [df_etapas_giro, df_etapas_tour, df_etapas_vuelta]):
    for _, row in df.iterrows():
        edicion_id = ediciones_map.get((carrera, int(row['edition'])))
        if edicion_id:
            cursor.execute("""
                INSERT INTO etapas (carrera, fecha, salida, llegada, distancia, tipo_etapa, metros_verticales,
                                    ganador_etapa, url_ganador, edad, nacionalidad, edicion_id)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """, (
                carrera,
                row['date'],
                row['departure'],
                row['arrival'],
                row['distance'] if pd.notna(row['distance']) else None,
                row['stage_type'],
                row['vertical_meters'] if pd.notna(row['vertical_meters']) else None,
                row['rider_name'],
                row['rider_url'],
                int(row['age']) if pd.notna(row['age']) else None,
                row['nationality'],
                edicion_id
            ))

# Insertar datos en puertos
for carrera, df in zip(["Giro", "Tour", "Vuelta"], [df_puertos_giro, df_puertos_tour, df_puertos_vuelta]):
    for _, row in df.iterrows():
        cursor.execute("""
            INSERT INTO puertos (carrera, puerto, num_etapas, num_ediciones, ano_primera_vez, latitud, longitud)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """, (
            carrera,
            row['Climb'],
            int(row['#stages']),
            int(row['#editions']),
            int(row['First year']),
            row['Latitud'] if pd.notna(row['Latitud']) else None,
            row['Longitud'] if pd.notna(row['Longitud']) else None
        ))

KeyError: 'edition'

In [ ]:
sbd.crear_basedatos("localhost","postgres","admin","5432","Ciclismo")

Error de tipo: la base de datos «Ciclismo» ya existe

